In [ ]:
# import python dependencies
import urllib2
import json
from datetime import datetime, timedelta
import csv
import time
import pandas as pd

In [ ]:
# Setting up credential

app_id = "1780311922214439"
app_secret = "42604fce105abb75cec9b5664882abbc"
# app_id dand app_secret can be generated by making facebook for developer account
# https://developers.facebook.com
# do not share app_id and app_secret to anyone
# mine had been reset at the time you read this notebook

access_token = app_id + "|" + app_secret

In [ ]:
def request_until_succeed(url):
    """
    : parameter
        url
    : return
        string
    """
    
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.now())

    return response.read()

def get_facebook_page_feed(page_id, access_token, num_status):
    """
    :parameters
        page_id (string) : name of page to be scraped
        access_token (string) : personal access token got from \
                                facebook for developer
        num_status (string) : number of status to be scraped
    
    :return
        list of dict
    """
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id," \
                 "likes.summary(true),"\
                 "comments.summary(true),"\
                 "shares&limit={0}&access_token={1}" \
                 .format(num_status, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data.get('data')
    
def process_facebook_page_feed(datum):
    """
    :parameters
        datum (dict) : single facebook feed
    
    :return
        dict
    """
    
    def cleanse_timestamp_format(ts):
        """
        :paramters: 
            timestamp (string)
        :return
            string
        """
        date = ts[0:10]
        time = ts[11:19]
        
        return date + ' ' + time

    message = datum.get('message')
    feed_type = datum.get('type')
    ts = str(datum.get('created_time'))
    creation_time = cleanse_timestamp_format(
                        str(datum.get('created_time'))
                    )
    if datum.get('likes') is not None:
        n_likes = datum.get('likes')\
                       .get('summary')\
                       .get('total_count')
    else:
        n_likes = 0
    if datum.get('comments') is not None:
        n_comments = datum.get('comments')\
                          .get('summary')\
                          .get('total_count')
    else:
        n_comments = 0
    if datum.get('shares') is not None:
        n_shared = datum.get('shares')\
                        .get('count')
    else:
        n_shared = 0
    link = datum.get('link')
    modified_at = datetime.now()\
                          .strftime("%Y-%m-%d %H:%M:%S")
        
    datum = {
        "message" : message,
        "feed_type" : feed_type,
        "creation_time" : creation_time,
        "n_likes" : n_likes,
        "n_comments" : n_comments,
        "n_shared" : n_shared,
        "link" : link,
        "modified_at" : modified_at
    }
    
    return datum

def facebook_page_feed_to_dataframe(feeds):
    """
    :parameters
        feed (list of dict)
    :return
        Dataframe Object
    """
    
    n = len(feeds)
    df = []
    for i in range(n):
        datum = process_facebook_page_feed(feeds[i])
        df.append(datum)
    
    return pd.DataFrame(df)

In [ ]:
page_id = "kompasTV"
data = get_facebook_page_feed(page_id, access_token, num_status=10)

In [ ]:
facebook_page_feed_to_dataframe(data)

,creation_time,feed_type,link,message,modified_at,n_comments,n_likes,n_shared
0,2016-11-13 03:15:45,video,https://www.facebook.com/1451508461541072/vide...,Setelah proses yang panjang dalam memilih peme...,2016-11-13 10:29:21,1,9,0
1,2016-11-13 02:36:00,photo,https://www.facebook.com/KompasTV/photos/a.196...,"Ayah, sosok tangguh tak butuh pujian. Ia tegas...",2016-11-13 10:29:21,1,171,18
2,2016-11-13 02:35:00,photo,https://www.facebook.com/KompasTV/photos/a.196...,Kangen penampilan para komika #SUCI6 seperti D...,2016-11-13 10:29:21,1,27,0
3,2016-11-13 02:22:00,photo,https://www.facebook.com/KompasTV/photos/a.196...,Ikuti terus program-program seru di KompasTV. ...,2016-11-13 10:29:21,32,47,2
4,2016-11-13 02:00:00,video,https://www.facebook.com/KompasTV/videos/13178...,Kepala Dinas Pendidikan Provinsi Sulawesi Sela...,2016-11-13 10:29:21,2,41,2
5,2016-11-13 01:00:00,video,https://www.facebook.com/KompasTV/videos/13156...,Jakarta Barrier Free Tourism (JBFT) adalah kom...,2016-11-13 10:29:21,2,89,13
6,2016-11-13 00:00:00,video,https://www.facebook.com/KompasTV/videos/13178...,Pemerintah kembali mengumumkan paket kebijakan...,2016-11-13 10:29:21,3,53,4
7,2016-11-12 23:00:00,video,https://www.facebook.com/KompasTV/videos/13171...,Al-Quran adalah kitab suci agama Islam yang wa...,2016-11-13 10:29:21,4,276,26
8,2016-11-12 17:02:03,link,http://internasional.kompas.com/read/2016/11/1...,"ISIS sendiri mengklaim, jumlah korban meningga...",2016-11-13 10:29:21,5,117,2
9,2016-11-12 16:00:00,video,https://www.facebook.com/KompasTV/videos/13173...,Oknum Bea dan Cukai Pelabuhan Tanjung Emas Sem...,2016-11-13 10:29:21,14,103,22
